In [ ]:
Oracle AI Data Platform v1.0

Copyright © 2025, Oracle and/or its affiliates.

Licensed under the Universal Permissive License v 1.0 as shown at https://oss.oracle.com/licenses/upl/

# Delta UniForm Notebook Samples

This notebook demonstrates how to create and work with **Delta UniForm tables** that synchronize Iceberg metadata, enabling interoperability with Iceberg clients.


## Create Delta UniForm Table with Iceberg Metadata Sync

The following example creates a Delta table and enables UniForm, so that Iceberg metadata is generated alongside Delta metadata.


In [ ]:
%sql
CREATE TABLE cat1.db1.uniform1 (
  id INT,
  name STRING,
  ts TIMESTAMP
)
USING DELTA
TBLPROPERTIES('delta.universalFormat.enabledFormats' = 'iceberg');

## Insert into Delta UniForm Table

Inserts are automatically reflected in both Delta and Iceberg metadata.


In [ ]:
%sql
INSERT INTO cat1.db1.uniform1 VALUES
(1, 'Alice', CURRENT_TIMESTAMP),
(2, 'Bob', CURRENT_TIMESTAMP);

## MERGE into Delta UniForm Table

MERGE statements also keep both metadata layers in sync.
The following example updates an existing record (id = 1) and inserts a new record (id = 3)

In [ ]:
%sql

-- Temporary table or inline source for upsert
CREATE OR REPLACE TEMP VIEW updates AS
SELECT * FROM VALUES
  (1, 'Alice_updated', current_timestamp()), -- Will UPDATE
  (3, 'Charlie', current_timestamp())        -- Will INSERT
AS updates(id, name, ts);

-- UPSERT via MERGE INTO
MERGE INTO cat1.db1.uniform1 AS target
USING updates AS source
ON target.id = source.id
WHEN MATCHED THEN UPDATE SET *
WHEN NOT MATCHED THEN INSERT *;